# Wetterdaten von 1950 bis (fast) heute: Analysieren

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## [1 Punkt] Einlesen des bereinigten Datensatzes

Lesen Sie den bereinigten Datensatz aus der Datei `produkt_tu_stunde_19500101_20221231_01639_bereinigt.csv` in einen `pd.DataFrame` ein. Dieser Datensatz basiert auf Daten des Deutschen Wetterdienstes, die dieser unter `https://opendata.dwd.de/` bereitstellt, wurde aber bereinigt, damit diese Aufgabe komplett unabhängig von der Aufgabe `Wetterdaten_vorbereiten` bearbeitet werden kann.

Insbesondere sollen Sie die Spalte `Datum` als Index mit Einträgen vom Typ `np.datetime64[ns]` einlesen. Nutzen Sie dazu die [`pd.read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)-Funktion  unter Angabe von `index_col` und `parse_dates=True`.

In [ ]:
# Bitte geben Sie Ihren Code hier ein.

## [2 Punkte] Probleme der Visualisierung mit Histogrammen und `value_counts`

Für die explorative Datenanalyse sind Histogramme und Plots der `value_counts` möglicherweise gut geeignet, aber zur Visualisierung nicht immer. Insbesondere können abhängig von der Datenquelle in Ihrer Darstellung Artefakte auftreten.

Stellen Sie die bereinigten Messwerte der Luftfeuchtigkeit als Histogramm, sowie die `value_counts` der bereinigten Temperaturdaten als Scatterplot dar. Variieren Sie die Anzahl der `bins` des Histogramms. Welche Artefakte fallen Ihnen jeweils auf? Welche Ursachen könnte dies haben?

In [ ]:
# Bitte geben Sie Ihren Code hier ein.

## [3 Punkte] Saisonalität der Temperatur und Luftfeuchtigkeit

Temperatur und Luftfeuchtigkeit sind *saisonale* Größen, sie ändern sich also einigermaßen vorhersagbar im Verlauf eines Jahres (auf der Nordhalbkugel ist es normalerweise im Juni wärmer als im Januar). Diese Saisonalität sollen Sie als erstes visualisieren. Beide Größen schwanken jedoch auch im Tagesverlauf — diese Periodizität interessiert uns hier aber nicht. Überlegen Sie insbesondere, wie Sie dies berücksichtigen.

Stellen Sie für die Monate Januar bis Dezember die *Verteilung* beider Größen über die Jahre hinweg in separaten Plots dar. Auf der Ordinate soll der Monat aufgetragen werden, auf der Abszisse die jeweilige Messgröße. Wählen Sie eine sinnvolle Darstellungsform, die nicht nur Mittelwert und Standardabweichung, sondern die tatsächliche Verteilung der Werte darstellt. Zum Beispiel könnten Sie einen Boxplot (`plt.boxplot`) oder einen Violinplot (`plt.violin`) nutzen.

*Hinweis*: Die `pd.DataFrame.groupby`-Methode könnte sich als nützlich erweisen.

In [ ]:
# Bitte geben Sie Ihren Code hier ein.

## [4 Punkte] Langfristige Veränderung der Jahrestemperatur

Infolge der Erhitzung der Erde in den letzten Jahrzehnten, die in weiten Teilen der Welt bereits katastrophale Auswirkungen zeigt, hat sich auch das Klima in Gießen verändert. Stellen Sie diese Veränderung grafisch dar. Dabei ist die genaue Art der Darstellung Ihnen überlassen. Lassen Sie sich gerne z.B. von [anderen Darstellungen](https://en.wikipedia.org/wiki/Climate_change_art) inspirieren.

In [ ]:
# Bitte geben Sie Ihren Code hier ein.